In [1]:
import numpy as np
import pandas as pd
import glob
from datetime import datetime
import xgboost as xgb
pd.options.display.max_columns = None

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/biens_2015-19.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765694 entries, 0 to 765693
Data columns (total 10 columns):
valeur_fonciere              765694 non-null float64
code_postal                  765694 non-null float64
commune                      765694 non-null object
code_departement             765694 non-null int64
type_local                   765694 non-null object
surface_reelle_bati          765694 non-null float64
nombre_pieces_principales    765694 non-null int64
surface_terrain              765694 non-null float64
year                         765694 non-null int64
month                        765694 non-null int64
dtypes: float64(4), int64(4), object(2)
memory usage: 58.4+ MB


## Encode categorical data

In [4]:
categories = ['code_departement', 'code_postal', 'type_local'] #, 'month', 'year'

In [5]:
for cat in categories:
    cat_val = df[cat].unique()
    print('{} ({}):'.format(cat,len(cat_val)))
    print('{}\n'.format(cat_val))

code_departement (97):
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  21  22  23  24  25  26  27  28  29 120 121  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  58  59  60  61  62  63  64  65  66  69  70  71  72  73  74
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95 971 972 973 974  75]

code_postal (5832):
[ 1160.  1960.  1290. ... 42111. 77157. 97116.]

type_local (2):
['Maison' 'Appartement']



In [6]:
df.drop('commune', axis=1, inplace=True)
df = pd.get_dummies(df, columns=categories)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765694 entries, 0 to 765693
Columns: 5937 entries, valeur_fonciere to type_local_Maison
dtypes: float64(3), int64(3), uint8(5931)
memory usage: 4.3 GB


In [8]:
#df.to_csv(r'data/ml_data.csv', index = None, header=True)

## XGBoost

In [9]:
y = df['valeur_fonciere']
X = df.drop(['valeur_fonciere'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.33, random_state=42, shuffle=True)


In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train,y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])

In [ ]:
preds = xlf.predict(X_test)

print('\nMean Square error" ', mean_squared_error(y_test,preds))